# Python based analysis

This script takes trees as inputs and writes another sets of trees as outputs.<br>
The trees are filtered in between.

In [1]:
import os, sys
import numpy as np
import uproot
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
#Global parameters

basedir = '../ROOT_FILES/treesWithNN/'
injob  = 'baseline/tree_2016preVFP_UL_baseline'
outjob = 'qcdcr/tree_2016preVFP_UL_topcr'
campaign = '2016preVFP_UL'
nnscore = 'nnscore_qcd_vlld_2016preVFP'

indir = os.path.join(basedir, injob)
outdir = os.path.join(basedir, outjob)
os.makedirs(outdir, exist_ok=True) 

In [3]:
import uproot
import pandas as pd
from tqdm.notebook import tqdm

def read_file_into_df(filepath, step_size=100000):
    with uproot.open(filepath) as tfile:
        if "myEvents" not in tfile:
            print(f"Warning: 'myEvents' tree not found in {filepath}. Skipping.")
            return pd.DataFrame()

        ttree = tfile["myEvents"]
        total_entries = ttree.num_entries

        if total_entries == 0:
            print(f"Warning: 'myEvents' tree in {filepath} is empty. Skipping.")
            return pd.DataFrame()

        total_batches = (total_entries + step_size - 1) // step_size
        dfs = []

        with tqdm(total=total_batches, unit=" batch", desc="Reading", colour='green', dynamic_ncols=False, mininterval=0.5) as pbar:
            for batch in ttree.iterate(ttree.keys(), step_size=step_size, library="pd"):
                dfs.append(batch)
                pbar.update(1)

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

def write_df_into_file(df, filepath, step_size=100000):
    if df.empty:
        print(f"Warning: Attempting to write empty DataFrame to {filepath}. Skipping.")
        return

    total_entries = len(df)
    total_batches = (total_entries + step_size - 1) // step_size

    with tqdm(total=total_batches, unit=" batch", desc="Reading", colour='green', dynamic_ncols=False, mininterval=0.5) as pbar:
        with uproot.recreate(filepath) as file:
            for start in range(0, total_entries, step_size):
                chunk = df.iloc[start:start + step_size]
                if start == 0: file["myEvents"] = {col: chunk[col].to_numpy() for col in df.columns}
                else:          file["myEvents"].extend({col: chunk[col].to_numpy() for col in df.columns})
                pbar.update(1)

print('Functions loaded.')

Functions loaded.


In [4]:
%%time

list_of_files = os.listdir(indir)
list_empty_input = []
list_empty_output = []

for f in list_of_files:

    #if not 'QCD' in f: continue
    if not f.endswith('.root'): continue
    #if 'SingleMuon' not in f: continue

    #Step1: Prepare the dataframe
    print(f'\nProcessing file: {f}')
    filepath = os.path.join(indir, f)
    sample = f.split("_")[1]
    subsample = "_".join(f.split("_")[2:])
       
    outfile = os.path.join(outdir, f)
    #if os.path.exists(outfile): continue
    #if 'WJets' not in f: continue

    df = read_file_into_df(filepath)
    if df.empty : 
        #print(f"\033[0;31mWarning: Empty input file: {filepath}\033[0m\n")
        list_empty_input.append(filepath)
        continue

    ####################################
    #         EVENT SELECTION          #
    ####################################

    #Step1: Controlling QCD:
    dy_veto = 'not (channel == 3 and 76 < dilep_mass < 106)'
    qcd_cr = f'{dy_veto} and {nnscore}<0.30 and 0.02<lep0_iso<0.15 and lep0_sip3d>5'
    qcd_vr = f'{dy_veto} and {nnscore}<0.30 and 0.02<lep0_iso<0.15 and lep0_sip3d<5'

    #Step2: Controlling Drell-Yan:
    dycr  = f'76<dilep_mass<106  and dilep_ptratio > 0.7'

    #Step3: Filter bad events:
    tight_sip3d =  'lep0_sip3d<5 and lep1_sip3d<10'
    good_events = f'{tight_sip3d} and {dy_veto} and {nnscore}>0.30 and HT>50' 
    
    #Step4: Controlling TTbar:
    top_cr = f'{good_events} and {nnscore}>0.70 and nbjet>0'    

    #Step5: Validation:
    val_region = f'{good_events} and 0.50<{nnscore}<0.70'

    #Step6: Signal regions:
    sr = f'{good_events} and {nnscore}>0.70 and nbjet==0'
    
    #------------------------------
    # Final event selection:
    event_selection = qcd_cr
    #------------------------------

    # Use df.query with the combined condition
    df_filtered = df.query(event_selection)
    
    if df_filtered.empty: list_empty_output.append(f'{sample}_{subsample}')

    nbefore = len(df)
    nafter = len(df_filtered)

    frac = 0
    if nbefore != 0: frac = nafter*100/nbefore

    #display(df_filtered)

    write_df_into_file(df_filtered, outfile)
    print(f'\033[1;32mFile written: {outfile}\033[0m ({nbefore} -> {nafter}, {frac:.2f}%)\n')
    #break #file

print(f'\n\033[1;33mSummary:\033[0m')
print(f'\n\033[33mEmpty input: {list_empty_input}\033[0m')
print(f'\n\033[31mEmpty output: {list_empty_output}\033[0m\n')


Processing file: tree_DYJetsToLL_M10to50.root


Reading:   0%|          | 0/1 [00:00<?, ? batch/s]

Reading:   0%|          | 0/1 [00:00<?, ? batch/s]

File written: ../ROOT_FILES/treesWithNN/qcdcr/tree_2017_UL_qcdcr/tree_DYJetsToLL_M10to50.root (172 -> 3, 1.74%)


Processing file: tree_DYJetsToLL_M50.root


Reading:   0%|          | 0/2 [00:00<?, ? batch/s]

...

Reading:   0%|          | 0/1 [00:00<?, ? batch/s]

File written: ../ROOT_FILES/treesWithNN/qcdcr/tree_2017_UL_qcdcr/tree_ZZ_ZZTo2Q2Nu.root (1 -> 0, 0.00%)


Processing file: tree_ZZ_ZZTo4L.root


Reading:   0%|          | 0/14 [00:00<?, ? batch/s]

Reading:   0%|          | 0/1 [00:00<?, ? batch/s]

File written: ../ROOT_FILES/treesWithNN/qcdcr/tree_2017_UL_qcdcr/tree_ZZ_ZZTo4L.root (1334237 -> 4478, 0.34%)


Summary:

Empty input: []

Empty output: ['QCD_EMEnriched_120to170.root', 'QCD_EMEnriched_170to300.root', 'QCD_EMEnriched_30to50.root', 'QCD_EMEnriched_50to80.root', 'QCD_EMEnriched_80to120.root', 'Rare_TTWZ.root', 'VLLD_ele_M1000.root', 'VLLD_ele_M300.root', 'VLLD_ele_M600.root', 'VLLD_mu_M200.root', 'VLLD_mu_M300.root', 'VLLD_mu_M400.root', 'VLLD_mu_M600.root', 'VLLD_mu_M800.root', 'VLLS_ele_M1000.root', 'VLLS_ele_M350.root', 'VLLS_ele_M450.root', 'VLLS_ele_M750.root', 'VLLS_mu_M450.root', 'VLLS_mu_M500.root', 'VLLS_mu_M750.root', 'WWZ_Inclusive.root', 'ZZZ_Inclusive.root', 'ZZ_ZZTo2Q2Nu.root']

CPU times: user 19.2 s, sys: 2.37 s, total: 21.6 s
Wall time: 38.5 s


In [5]:
%%time

run_here = False

channels = ['mm', 'me', 'em', 'ee']
for channel_ in channels:
    jobname_ = outjob
    campaign_ = campaign
    dump_ = outjob.replace('tree', 'hist')+'_'+channel_
    arguments = f'extractHistsFromTrees.C("{jobname_}", "{dump_}", "{campaign_}", "{channel_}")'
    command = f"root -q -b -l '{arguments}'"
    print(f'\033[93m\nRunnning .. \n{command}\033[0m\n')

    if run_here: os.system(command)
    else: print('Run it yourself:')
    
print('Done!')


Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("qcdcr/tree_2017_UL_qcdcr", "qcdcr/hist_2017_UL_qcdcr_mm", "2017_UL", "mm")'

Run it yourself:

Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("qcdcr/tree_2017_UL_qcdcr", "qcdcr/hist_2017_UL_qcdcr_me", "2017_UL", "me")'

Run it yourself:

Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("qcdcr/tree_2017_UL_qcdcr", "qcdcr/hist_2017_UL_qcdcr_em", "2017_UL", "em")'

Run it yourself:

Runnning .. 
root -q -b -l 'extractHistsFromTrees.C("qcdcr/tree_2017_UL_qcdcr", "qcdcr/hist_2017_UL_qcdcr_ee", "2017_UL", "ee")'

Run it yourself:
Done!
CPU times: user 146 μs, sys: 17 μs, total: 163 μs
Wall time: 160 μs
